In [1]:
import pandas as pd
import plotly.express as px
import glob
from collections import defaultdict
import re
import datetime
# import wget
import time
import requests

In [ ]:
## apparently, pandas is not enough to read excel files
## had to install openpyxl also
## run only once
# !pip3 install openpyxl


In [ ]:
# download data programmatically. This is only run once
base_sector_url = "https://portal.census.gov/pulse/data/downloads/10/national_state_sector_09Aug20_15Aug20.xlsx"
base_msa_url = "https://portal.census.gov/pulse/data/downloads/%s/top_50_msa_%s_%s.xlsx"

def download_census(base_url, start_date, i):
    end = start_date + datetime.timedelta(days=6)
    file_url = base_msa_url %(i, start.strftime("%d%b%y"), end.strftime("%d%b%y"))
    local_file = "./raw_data/census_pulse_msa_%s_%s.xlsx" %(start.strftime("%d%b%y"), end.strftime("%d%b%y"))
    print("Trying ", file_url)
    try:
        resp = requests.get(file_url)
        with open(local_file, 'wb') as output:
            output.write(resp.content)
        time.sleep(3)
    except:
        pass

## there is a 4-5 week break between survey instruments. We have to skip those dates otherwise they 
## will mess up the autoincrementing number
skip = ['2020-06-28', '2020-07-05', '2020-07-12', '2020-07-19', '2020-07-26', '2020-08-02', '2020-10-11', 
        '2020-10-18', '2020-10-25', '2020-11-01', '2021-01-11', '2021-01-18', '2021-01-25', '2021-02-01', 
        '2021-02-08', '2021-04-19', '2021-04-26', '2021-05-03', '2021-05-10', '2021-07-19', '2021-07-26', 
        '2021-08-02', '2021-08-09']
skip_dates = [datetime.datetime.strptime(dt, "%Y-%m-%d").date() for dt in skip]

    
# first_survey_date_2020 = datetime.datetime(2020, 4, 26).date()
# last_survey_date_2020 = datetime.datetime(2020, 12, 29).date()

# start = first_survey_date_2020
# i = 1

# while start < last_survey_date_2020:
#     if start not in skip_dates:
#         download_census(base_msa_url, start, i)
#         start = start + datetime.timedelta(days=7)
#         i += 1 
#     else:
#         start = start + datetime.timedelta(days=7)

## in 2020 survey weeks started on Sundays, however in 2021 they start on Mondays so we have to split it up. 
## The auto-incremented number continues, though.

first_survey_date_2021 = datetime.datetime(2021, 1, 4).date()
last_survey_date_2021 = datetime.datetime(2021, 9, 21).date()

start = first_survey_date_2021
i = 27
while start < last_survey_date_2021:
    if start not in skip_dates:
        download_census(base_msa_url, start, i)
        start = start + datetime.timedelta(days=7)  
        i += 1
    else:
        start = start + datetime.timedelta(days=7)



In [2]:
## set global variables
yelp_cities = ['Montreal', 'Calgary', 'Toronto', 'Pittsburgh', 'Charlotte', 'Urbana-Champaign', 'Phoenix',
              'Las Vegas', 'Madison', 'Cleveland']
yelp_states = ['PA', 'NC', 'IL', 'AZ', 'NV', 'WI', 'OH']

# read in files using glob
dpath = "/Users/christinabrady/Documents/codebase/ds4a_women_project/raw_data/*.xlsx"
msas = glob.glob(dpath)

# get MSAs that match yelp data
matches = defaultdict(list)
census_cities = pd.read_excel(msas[0]).MSA.unique()
for ycity in yelp_cities:
    for ccity in census_cities:
        if ycity in ccity:
            matches[ycity].append(ccity)
    
flattened_ccities = [city for cities in matches.values() for city in cities]       
    

In [3]:
print("There are", len(matches), "matching cities.")
print(matches.keys())
print(matches.values())

There are 5 matching cities.
dict_keys(['Pittsburgh', 'Charlotte', 'Phoenix', 'Las Vegas', 'Cleveland'])
dict_values([['Pittsburgh, PA MSA'], ['Charlotte-Concord-Gastonia, NC-SC MSA'], ['Phoenix-Mesa-Chandler, AZ MSA'], ['Las Vegas-Henderson-Paradise, NV MSA'], ['Cleveland-Elyria, OH MSA']])


In [4]:
## helper functions
def read_msa_data(fl, locations, question, response):
    """ takes a file name and a list of MSAs, 
    a question (Instrument ID) and answer or list of answers (Answer ID)
    and returns a data frame of the percentage of businesses
    that gave the chosen responses to the chosen questions in the survey
    for that week
    """
    tmpdf = pd.read_excel(fl)
    
    # make the column names all lower case
    tmpdf.columns = tmpdf.columns.str.lower()
    
    # set the index to the MSA, which will make it easier to subset
    tmpdf = tmpdf.set_index(['msa'])
    
    locations_of_interest = tmpdf.loc[locations]
    ret = locations_of_interest[locations_of_interest.instrument_id.isin(question) & 
                                locations_of_interest.answer_id.isin(response)]
    return(ret)

# test
# test_run = read_census_data(msas[0], flattened_ccities, 1, [1, 2])
# test_run.head()

def get_date(fl, which_date="end"):
    survey_dates = re.search(r"([0-9]{2}[A-Z][a-z].[0-9]{2})_([0-9]{2}[A-Z][a-z].[0-9]{2})", fl)
    if which_date == "start":
        dt = survey_dates.group(0)
    elif which_date == "end":
        dt = survey_dates.group(1)
    else:
        sys.exit("Please enter 'start' or 'end'.")
    return(datetime.datetime.strptime(dt, "%d%b%y").date())

## test getdate
# get_date(msas[0])



In [ ]:
# tmpdf = pd.read_excel(sectors[0])
    
# # make the column names all lower case
# tmpdf.columns = tmpdf.columns.str.lower()
# tmpdf.naics_sector = tmpdf.naics_sector.str.strip()
# tmpdf.head()

# # tmpdf.naics_sector.unique()
# # tmpdf.st.unique()
# # tmpdf.empclass.unique()
# # tmpdf.employee_size.unique()

# tmpdf = tmpdf.set_index(['st'])
# # # tmpdf.head()
# locations_of_interest = tmpdf.loc[yelp_states]
# # # ret = locations_of_interest[locations_of_interest.instrument_id == 1 & 
# # #                             locations_of_interest.answer_id.isin([1,2]) & 
# # #                             locations_of_interest.naics_sector == '72']

# # locations_of_interest.head()
# restaurants = tmpdf[tmpdf.naics_sector == '72']
# restaurants.index

In [5]:
### put it together and plot it
msa_data = []
for fl in msas:
    tmp = read_msa_data(fl, flattened_ccities, [2], [1, 2])
    tmp["survey_end_date"] = get_date(fl)
    msa_data.append(tmp)
    
all_msa_data = pd.concat(msa_data)
all_msa_data.dtypes

## convert the pecentages to a float
## so that we can combine them and plot them
all_msa_data["estimate_percentage_num"] = all_msa_data.estimate_percentage.str.replace("%", "")
all_msa_data["estimate_percentage_num"] = all_msa_data.estimate_percentage_num.str.strip()
all_msa_data["estimate_percentage_num"] = pd.to_numeric(all_msa_data.estimate_percentage_num)
all_msa_data.head()




,cbsa_code,instrument_id,question,answer_id,answer_text,estimate_percentage,se,survey_end_date,estimate_percentage_num
msa,,,,,,,,,
"Pittsburgh, PA MSA",38300,2,"Overall, how has this business been affected b...",1,Large negative effect,29.3%,4.47%,2021-09-06,29.3
"Pittsburgh, PA MSA",38300,2,"Overall, how has this business been affected b...",2,Moderate negative effect,35.3%,4.74%,2021-09-06,35.3
"Charlotte-Concord-Gastonia, NC-SC MSA",16740,2,"Overall, how has this business been affected b...",1,Large negative effect,25.2%,4.50%,2021-09-06,25.2
"Charlotte-Concord-Gastonia, NC-SC MSA",16740,2,"Overall, how has this business been affected b...",2,Moderate negative effect,33.7%,5.69%,2021-09-06,33.7
"Phoenix-Mesa-Chandler, AZ MSA",38060,2,"Overall, how has this business been affected b...",1,Large negative effect,21.7%,4.11%,2021-09-06,21.7


In [7]:
## first let's look at only a large negative effect
msa_large_negative = all_msa_data[all_msa_data.answer_id == 1]
msa_large_negative = msa_large_negative.reset_index(drop = False)

## estimate percentage is read in as a string. We need to convert it to a float
## so that we can graph it properly

msa_large_negative = msa_large_negative.sort_values(["msa", "survey_end_date"])

fig1 = px.line(msa_large_negative, 
              x="survey_end_date", 
              y="estimate_percentage_num", 
              color="msa", 
              line_group="msa", 
              hover_name="msa",
              line_shape="spline", 
              render_mode="svg",
             markers = True)
fig1.update_yaxes(range=[0,100])
fig1.show()

That is all over the place, which may make sense since this was the beginning of the pandemic. We can look at more data later. Right now, let's look at the combined negative affect. 

In [8]:
all_negative = all_msa_data[["survey_end_date", "estimate_percentage_num"]]
all_negative = all_negative.groupby(["msa", "survey_end_date"]).sum().reset_index()
fig2 = px.line(all_negative, 
              x="survey_end_date", 
              y="estimate_percentage_num", 
              color="msa", 
              line_group="msa", 
              hover_name="msa",
              line_shape="spline", 
              render_mode="svg",
             markers = True)
fig2.update_yaxes(range=[0,100])
fig2.show()